# Some basic viz with matplotlib 📊 📈

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pyrsm as rsm

---

In [2]:
df = pd.read_csv("../Data/Final_Vader_Time.csv")

df.shape, df.columns

/var/folders/79/jg9qrvxj5v1_ft3s9qfkpbl40000gn/T/ipykernel_79503/3546707197.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Data/Final_Vader_Time.csv")


((231306, 19),
 Index(['Author', 'Timestamp', 'Comment', 'Article', 'Game Title', 'Console',
        'Commenter Name', 'Reviewer Type', 'Date Commented', 'Score Number',
        'Release Date', 'Description', 'Game Link', 'Article Link', 'Platform',
        'Title', 'Upvotes', 'Vader_Score', 'Correct_TimeStamp'],
       dtype='object'))

In [3]:
df.head()

,Author,Timestamp,Comment,Article,Game Title,Console,Commenter Name,Reviewer Type,Date Commented,Score Number,Release Date,Description,Game Link,Article Link,Platform,Title,Upvotes,Vader_Score,Correct_TimeStamp
0,shaftedman,1 day ago,"IGN Plus... just out of curiosity, does anyone...",daily-deals-the-last-of-us-part-i-spider-man-m...,GameSwitch Sports,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,2024-01-05 00:00:00
1,gamingdevil800,"3 July, 2021",This would be a lot more simplistic if you jus...,video-game-release-dates-ps4-ps5-xbox-one-seri...,GameFinal Fantasy 7 Rebirth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,2021-07-03 00:00:00
2,Snagglepuss000,"1 June, 2021",Back 4 Blood is NOT releasing in June. Pushed ...,video-game-release-dates-ps4-ps5-xbox-one-seri...,GameFinal Fantasy 7 Rebirth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral,2021-06-01 00:00:00
3,seebs,"22 July, 2021","MS Flight Simulator, The Ascent, Hades, 12 Min...",video-game-release-dates-ps4-ps5-xbox-one-seri...,GameFinal Fantasy 7 Rebirth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral,2021-07-22 00:00:00
4,LedZepp9,"1 June, 2021","All the comments on ""there isn't any games, no...",video-game-release-dates-ps4-ps5-xbox-one-seri...,GameFinal Fantasy 7 Rebirth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,2021-06-01 00:00:00
